# CONTENT BASED MODEL TO RECOMMEND A MOVIES TO A USER

## Loading the important library for the model

In [1]:
from __future__ import print_function
import pandas as pd
import re
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_columns = 80

2023-06-05 00:34:11.282572: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-05 00:34:11.282600: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


### Loading the data 

In [2]:
Ratingsdf =  pd.read_csv ("Netflix_Dataset_Rating.csv")
Ratingsdf.head()

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3


In [3]:
Moviesdf = pd.read_csv ("Netflix_Dataset_Movie.csv")
Moviesdf.head()

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


In [4]:
# Create a merged datafarme that has both the movies and ratings datasets
Movies_DF = Moviesdf.merge(Ratingsdf, on='Movie_ID')
Movies_DF.head()

,Movie_ID,Year,Name,User_ID,Rating
0,3,1997,Character,712664,5
1,3,1997,Character,1331154,4
2,3,1997,Character,2632461,3
3,3,1997,Character,44937,5
4,3,1997,Character,656399,4


In [5]:
MoviesInfo_df = pd.read_csv ("imdb_processed.csv")
MoviesInfo_df.head()

,Name,Year,kind,genre,imdb_rating,vote,country,language,cast,director,composer,writer,runtime
0,Dinosaur Planet,2003,tv mini series,"['Documentary', 'Animation', 'Family']",7.7,474.0,['United States'],['English'],"['Christian Slater', 'Scott Sampson']",NaN,NaN,"['Mike Carrol', 'Mike Carroll', 'Georgann Kane']",50.0
1,Get Up and Dance!,1994,video movie,['Family'],8.1,18.0,['United States'],['English'],"['Paula Abdul', 'Aurorah Allain', 'Bill Bohl',...",['Steve Purcell'],NaN,NaN,83.0
2,8 Man,1992,movie,"['Action', 'Sci-Fi']",5.5,93.0,['Japan'],['Japanese'],"['Kai Shishido', 'Etsushi Takahashi', 'Sachiko...",['Yasuhiro Horiuchi'],['Carole King'],"['Kazumasa Hirai', 'Jirô Kuwata', 'Junko Suzuki']",NaN
3,What the #$*! Do We (K)now!?,2004,movie,"['Documentary', 'Comedy', 'Drama', 'Fantasy', ...",5.3,13432.0,['United States'],"['English', 'German', 'Spanish']","['Marlee Matlin', 'Elaine Hendrix', 'John Ross...","['William Arntz', 'Betsy Chasse', 'Mark Vicente']",['Christopher Franke'],"['William Arntz', 'Betsy Chasse', 'Matthew Hof...",60.0
4,Class of Nuke 'Em High Part II: Subhumanoid Me...,1991,movie,"['Comedy', 'Horror', 'Sci-Fi']",4.5,2177.0,['United States'],['English'],"['Brick Bronsky', 'Lisa Gaye', 'Leesa Rowland'...",['Eric Louzil'],['Bob Mithoff'],"['Lloyd Kaufman', 'Carl Morano', 'Matt Unger']",96.0


In [6]:
## Merge the movies data frame with the movies info data frame so that it creates an big data frame 
##with both the ratings and description information of the movies. 
# Create one merged DataFrame containing all the Netflix data.
MoviesDF = Moviesdf.merge(Ratingsdf, on='Movie_ID').merge(MoviesInfo_df, on=['Name','Year'])
MoviesDF.head()

,Movie_ID,Year,Name,User_ID,Rating,kind,genre,imdb_rating,vote,country,language,cast,director,composer,writer,runtime
0,17,2005,7 Seconds,2187374,4,video movie,"['Action', 'Comedy', 'Crime', 'Thriller']",4.8,7153.0,"['United Kingdom', 'Romania', 'Switzerland', '...",['English'],"['Wesley Snipes', 'Tamzin Outhwaite', 'Deobia ...",['Simon Fellows'],"['Barry Taylor', 'Neal Acree']",['Martin Wheeler'],118.0
1,17,2005,7 Seconds,2503129,5,video movie,"['Action', 'Comedy', 'Crime', 'Thriller']",4.8,7153.0,"['United Kingdom', 'Romania', 'Switzerland', '...",['English'],"['Wesley Snipes', 'Tamzin Outhwaite', 'Deobia ...",['Simon Fellows'],"['Barry Taylor', 'Neal Acree']",['Martin Wheeler'],118.0
2,17,2005,7 Seconds,263315,3,video movie,"['Action', 'Comedy', 'Crime', 'Thriller']",4.8,7153.0,"['United Kingdom', 'Romania', 'Switzerland', '...",['English'],"['Wesley Snipes', 'Tamzin Outhwaite', 'Deobia ...",['Simon Fellows'],"['Barry Taylor', 'Neal Acree']",['Martin Wheeler'],118.0
3,17,2005,7 Seconds,608309,3,video movie,"['Action', 'Comedy', 'Crime', 'Thriller']",4.8,7153.0,"['United Kingdom', 'Romania', 'Switzerland', '...",['English'],"['Wesley Snipes', 'Tamzin Outhwaite', 'Deobia ...",['Simon Fellows'],"['Barry Taylor', 'Neal Acree']",['Martin Wheeler'],118.0
4,17,2005,7 Seconds,2336678,2,video movie,"['Action', 'Comedy', 'Crime', 'Thriller']",4.8,7153.0,"['United Kingdom', 'Romania', 'Switzerland', '...",['English'],"['Wesley Snipes', 'Tamzin Outhwaite', 'Deobia ...",['Simon Fellows'],"['Barry Taylor', 'Neal Acree']",['Martin Wheeler'],118.0


In [7]:
## Remove the columns that you wont be considering for the recommendation system.
NewMoviesDF = MoviesDF.drop(columns=['kind','vote','country','cast','composer','runtime'])
NewMoviesDF.head()

,Movie_ID,Year,Name,User_ID,Rating,genre,imdb_rating,language,director,writer
0,17,2005,7 Seconds,2187374,4,"['Action', 'Comedy', 'Crime', 'Thriller']",4.8,['English'],['Simon Fellows'],['Martin Wheeler']
1,17,2005,7 Seconds,2503129,5,"['Action', 'Comedy', 'Crime', 'Thriller']",4.8,['English'],['Simon Fellows'],['Martin Wheeler']
2,17,2005,7 Seconds,263315,3,"['Action', 'Comedy', 'Crime', 'Thriller']",4.8,['English'],['Simon Fellows'],['Martin Wheeler']
3,17,2005,7 Seconds,608309,3,"['Action', 'Comedy', 'Crime', 'Thriller']",4.8,['English'],['Simon Fellows'],['Martin Wheeler']
4,17,2005,7 Seconds,2336678,2,"['Action', 'Comedy', 'Crime', 'Thriller']",4.8,['English'],['Simon Fellows'],['Martin Wheeler']


In [8]:
# Preprocess the data
## this process is to help isolate the different genres of the movies. 
for i, row in NewMoviesDF.iterrows():
    genres = row['genre']
    
    # Handle the case when genre is a string
    if isinstance(genres, str):
        genres = genres.strip('[]').replace("'", "").split(', ')
        NewMoviesDF.at[i, 'genre'] = genres
    
    # Handle the case when genre is a list of strings
    if isinstance(genres, list):
        genres = [genre.strip() for genre in genres]
        NewMoviesDF.at[i, 'genre'] = genres

In [9]:
# Remove rows with 'Unknown' genre
NewMoviesDF = NewMoviesDF[~NewMoviesDF['genre'].apply(lambda x: 'Unknown' in x)]

In [10]:
# Check the unique genres again
unique_genres = NewMoviesDF['genre'].explode().unique()
print(unique_genres)

['Action' 'Comedy' 'Crime' 'Thriller' 'Drama' 'Romance' 'Documentary'
 'History' 'War' 'Music' 'Sci-Fi' 'Adventure' 'Horror' 'Western' 'Fantasy'
 'Family' 'Mystery' 'Biography' 'Animation' 'Sport' 'Musical' 'Film-Noir'
 'Short']


### Candidate Generation 

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from joblib import Parallel, delayed
##from sklearn.decomposition import TruncatedSVD

In [12]:
##TF-IDF will transform the text into meaningful representation of integers or numbers 
##which is used to fit machine learning algorithm for predictions.
# Create a TF-IDF vectorizer
#vectorizer = TfidfVectorizer()

In [13]:
from scipy import sparse
# Fit and transform the movie genres
##The TF-IDF vectorizer creates a matrix that can be quite large, especially given the a large number of unique genres.
##I attempted using the scipy.sparse module to store the genre matrix as a sparse matrix,
##which can save memory and improve performance.
# Convert the genre list to string
NewMoviesDF['genre'] = NewMoviesDF['genre'].apply(lambda x: ' '.join(x))

# Fit and transform the movie genres
vectorizer = TfidfVectorizer()
genre_matrix = sparse.csr_matrix(vectorizer.fit_transform(NewMoviesDF['genre']))

In [14]:
# Create a mapping of movie titles to their corresponding indices
title_indices = pd.Series(NewMoviesDF.index, index=NewMoviesDF['Name']).drop_duplicates()

In [15]:
from sklearn.neighbors import KDTree
# Convert the sparse matrix to a dense numpy array
dense_genre_matrix = genre_matrix.toarray()
# Build the KDTree
kdtree = KDTree(dense_genre_matrix)

In [16]:
##Optimize cosine similarity calculation: The cosine similarity calculation can 
##be computationally expensive, especially for large matrices. 
##I can optimize this step by using by using matrix factorization techniques like
##singular value decomposition (SVD) to reduce the dimensionality of the genre matrix.
# Reduce dimensionality using SVD
#svd = TruncatedSVD(n_components=min(genre_matrix.shape[1]-1, 100))
#reduced_genre_matrix = svd.fit_transform(genre_matrix)

In [17]:
# Define a function to calculate cosine similarity scores in parallel
#def calculate_scores(idx):
    #return cosine_similarity(reduced_genre_matrix[idx].reshape(1, -1), reduced_genre_matrix).flatten()

In [18]:
def calculate_scores(movie_index, top_n_neighbors):
    query_point = dense_genre_matrix[movie_index]
    distances, indices = kdtree.query(query_point.reshape(1, -1), k=top_n_neighbors+1)
    indices = indices[0][1:]  # Exclude the query movie itself
    return indices

In [19]:
from joblib import Parallel, delayed
from numpy import vstack

# Define the get_recommendations function
def get_recommendations(Name, top_n=5):
    idx = title_indices[Name]

    # Utilize parallel processing to calculate similarity scores
    scores = Parallel(n_jobs=-1)(delayed(calculate_scores)(i, top_n) for i in range(len(NewMoviesDF)))

    # Get the indices of the top-n similar movies
    top_indices = vstack(scores).argsort(axis=0)[:top_n].flatten()

    # Get the titles of the recommended movies
    recommended_movies = NewMoviesDF['Name'].iloc[top_indices].values

    return recommended_movies

In [20]:
# Example usage
recommended_movies = get_recommendations('7 Seconds', top_n=5)
print(recommended_movies)

['7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds'
 '7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds'
 '7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds'
 '7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds' '7 Seconds'
 '7 Seconds']


### Scoring

### Calculating the similarity scores between movies based on their genre using cosine similarity.

#### Extract a user from your datafarme and use that as a user profile that details their prefereed movies 

##### Theresa profile 

In [21]:
Theresa_profile = NewMoviesDF[NewMoviesDF['User_ID'] == 2503129	]
Theresa_profile

,Movie_ID,Year,Name,User_ID,Rating,genre,imdb_rating,language,director,writer
1,17,2005,7 Seconds,2503129,5,Action Comedy Crime Thriller,4.8,['English'],['Simon Fellows'],['Martin Wheeler']
10028,165,1982,Richard Pryor: Live on the Sunset Strip,2503129,3,Comedy,7.7,['English'],['Joe Layton'],"['Paul Mooney', 'Richard Pryor']"
20842,191,2003,X2: X-Men United,2503129,4,Action Sci-Fi Thriller,7.4,"['English', 'German', 'Italian', 'Spanish']",['Bryan Singer'],"['Zak Penn', 'David Hayter', 'Bryan Singer']"
158687,489,1998,Return to Paradise,2503129,4,Drama Romance Thriller,6.9,['English'],['Joseph Ruben'],"['Pierre Jolivet', 'Olivier Schatzky', 'Wesley..."
176921,692,1992,The Hand that Rocks the Cradle,2503129,4,Drama Thriller,6.6,['English'],['Curtis Hanson'],['Amanda Silver']
356117,1148,1981,For Your Eyes Only,2503129,3,Action Adventure Thriller,6.7,"['English', 'Greek', 'Italian', 'Spanish']",['John Glen'],"['Richard Maibaum', 'Michael G. Wilson', 'Ian ..."
383658,1202,1983,National Lampoon's Vacation,2503129,5,Adventure Comedy,7.4,['English'],['Harold Ramis'],"['John Hughes', 'John Hughes']"
499701,1425,1987,No Way Out,2503129,5,Action Crime Drama Mystery Romance Thriller,7.1,"['English', 'Russian']",['Roger Donaldson'],"['Kenneth Fearing', 'Robert Garland', 'Robert ..."
499702,1425,1987,No Way Out,2503129,5,Action Crime Drama Mystery Romance Thriller,7.1,"['English', 'Russian']",['Roger Donaldson'],"['Kenneth Fearing', 'Robert Garland', 'Robert ..."
629705,1905,2003,Pirates of the Caribbean: The Curse of the Bla...,2503129,5,Action Adventure Fantasy,8.0,['English'],['Gore Verbinski'],"['Ted Elliott', 'Terry Rossio', 'Stuart Beattie']"


In [ ]:
# Calculate the cosine similarity scores
scores = cosine_similarity(dense_genre_matrix)

In [ ]:
# Example usage
movie1_idx = title_indices['Return to Paradise']
movie2_idx = title_indices['The Hand that Rocks the Cradle']
similarity_score = scores[movie1_idx, movie2_idx]
print(similarity_score)

### Re-ranking

In [ ]:
## ranking the recommended movies based on their similarity scores.
# Define a function to rank recommended movies by similarity score
def rank_movies(recommended_movies, title):
    # Get the indices of the recommended movies
    indices = [title_indices[movie] for movie in recommended_movies]

    # Get the similarity scores of the recommended movies
    scores = scores[title_indices[title], indices]

    # Create a DataFrame with movie titles and their scores
    ranked_movies = pd.DataFrame({'Name': recommended_movies, 'score': scores})

    # Sort the DataFrame by score in descending order
    ranked_movies = ranked_movies.sort_values('score', ascending=False)

    return ranked_movies